# Réseaux Navigocorpus

In [20]:
import re
import csv
import networkx as nx
from poitousprint import Portic

In [2]:
INNER_ADMIRALTIES = {'La Rochelle', 'Marennes', 'Sables-d’Olonne'}
INNER_PROVINCES = {'Aunis', 'Poitou', 'Saintonge'}

In [3]:
client = Portic()

In [4]:
# Ancienne méthode
# flows = client.get_flows(year=1789, source_subset='Poitou_1789')
# flows = [flow for flow in flows if flow['departure_action'] == 'Out']

In [15]:
# En utilisant les données ajustées
with open('./resources/flows.csv') as f:
    flows = list(csv.DictReader(f))
    
    for f in flows:
        f['departure'] = f['port_depart']
        f['destination'] = f['port_destination']

In [21]:
SPLITTER = re.compile(r'\s*(?:et|-)')
tonnage_classes = sorted(set(flow['tonnage_class'] for flow in flows if flow['tonnage_class']), key=lambda x: int(SPLITTER.split(x[1:])[0]))
tonnage_classes

['[1-20]', '[21-50]', '[51-100]', '[101-200]', '[201-500]', '[501 et plus]']

## Réseau macro

In [34]:
def build_graph(name, tonnage_class=None, inner=False):
    graph = nx.DiGraph()

    def add_node(g, name, admiralty=None, peche=0):
        if admiralty is None:
            admiralty = 'n/a'

        if g.has_node(name):
            g.nodes[name]['peche'] += peche
        else:
            g.add_node(
                name,
                admiralty=admiralty,
                peche=peche,
                in_region=admiralty in INNER_ADMIRALTIES,
                inside_degree=0,
                outside_degree=0
            )

    def add_edge(g, source, target, tonnage):
        if g.has_edge(source, target):
            attr = g[source][target]
            attr['weight'] += 1
            attr['tonnage'] += tonnage
        else:
            g.add_edge(
                source,
                target,
                weight=1,
                tonnage=tonnage
            )

    for flow in flows:
        if tonnage_class is not None and flow['tonnage_class'] != tonnage_class:
            continue
        
        source = flow['departure']
        target = flow['destination']

        source_admiralty = flow['departure_admiralty']
        target_admiralty = flow['destination_admiralty']
        
        tonnage = 0

        try:
            tonnage = int(flow['tonnage'])
        except ValueError:
            pass

        inside_flow = source_admiralty in INNER_ADMIRALTIES and target_admiralty in INNER_ADMIRALTIES
        
        if inner and not inside_flow:
            continue

        # Macro graph
        if source == target:
            add_node(graph, source, source_admiralty, 1)
        else:
            add_node(graph, source, source_admiralty)
            add_node(graph, target, target_admiralty)
            add_edge(graph, source, target, tonnage)

            if source_admiralty in INNER_ADMIRALTIES:
                if inside_flow:
                    graph.nodes[source]['inside_degree'] += 1
                else:
                    graph.nodes[source]['outside_degree'] += 1
    
    nx.write_gexf(graph, './outputs/%s.gexf' % name)

In [35]:
build_graph('macro')
build_graph('inner', inner=True)

In [ ]:
for cls in tonnage_classes:
    build_graph(cls, tonnage_class=cls)